In [1]:
import json
import numpy as np

Read all of the data and format it for further processing.

In [3]:
from collections import defaultdict
from collections import Counter
f =open("../reviews_Grocery_and_Gourmet_Food.json",mode='r')
L = f.readlines()
review_list=[]
product_id = []
d=defaultdict(list)
review_dict_list=[]
c=0
for l in L:
    dd={}
    jj = json.loads(l)
    product_id+=[jj['asin']]
    #if jj["asin"] == "B00LQWKDBM":
    d[jj['asin']].append(jj['reviewText'])
    if (len(jj['reviewText'].split(" "))>12):
        review_list += [jj['reviewText']] 
    
    dd["item_id"] = str(c)
    dd["content"] = str(jj['reviewText'])
    c+=1
    dd_json=json.dumps(dd)
    review_dict_list+=[dd_json]
f.close()

In [28]:
#text preprocessing
import re

def removePunctuation(text):
    text=text.lower().strip()
    text=re.sub("[^0-9a-zA-Z ]", "", text)
    return text 



In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy
import scipy as sp
import re

import urllib2

from sklearn.cluster import KMeans
import nltk.stem
from sklearn import decomposition
import unicodedata



Topic Modeling with Non-negative Matrix Factorization.


In [8]:
global english_stemmer 
english_stemmer = nltk.stem.SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

class TopicAnalysis(object):
    def Vectorize(self,overviewlist):
        
        self.vectorizer = TfidfVectorizer(min_df=18, max_df=1.0,stop_words = 'english')
        self.vectorized = self.vectorizer.fit_transform(overviewlist)


    def NMF(self):

        n_topics = 30
        n_top_words = 20
        self.nmf = decomposition.NMF(n_components=n_topics).fit(self.vectorized)

        self.feature_names = self.vectorizer.get_feature_names()

        self.topic=[]
        self.topic_weight=[]
        for topic_idx, topic in enumerate(self.nmf.components_):

            self.topic+=[ " ".join([self.feature_names[i] \
                                    for i in topic.argsort()[:-n_top_words - 1:-1]])	]
            self.topic_weight+=[[topic[i] for i in topic.argsort()[:-n_top_words - 1:-1]]]
        self.weight_mat=self.nmf.fit_transform(self.vectorized)

        self.topic_id=[]
        for i in xrange(len(self.weight_mat)):
            idx=numpy.argsort(self.weight_mat[i])
            self.topic_id+=[idx]

In [29]:
TT = TopicAnalysis()
TT.Vectorize(review_list[0:10000])
TT.NMF()

In [24]:
review_list[0]

u"No sugar, no GMO garbage, no fillers that come with store bought extracts. This stuff is just amazing. I use it in everything from baking to cooking and even as suggested in my coffee which is saying a lot because I normally do not care for flavored coffee! You cannot go wrong with this. I've ordered from this merchant before, customer satisfaction is their priority and service was quick, shipped right out with tracking even! I'll be buying from GLS Goods again! I won't use any other vanilla!"

In [30]:
map(lambda x:set(removePunctuation(review_list[0]).lower().split(" ")).intersection(set(x.split(" ")).intersection()),TT.topic)

[set(),
 set(),
 {u'quick', u'service'},
 {u'coffee'},
 {u'cooking'},
 {u'sugar'},
 set(),
 set(),
 set(),
 {u'amazing', u'just', u'right'},
 set(),
 {u'amazing', u'vanilla'},
 set(),
 set(),
 set(),
 set(),
 {u'bought'},
 {u'lot'},
 set(),
 {u'cooking'},
 {u'amazing', u'buying'},
 {u'stuff'},
 {u'bought', u'buying', u'store'},
 {u'bought'},
 {u'ordered'},
 set(),
 set(),
 set(),
 {u'use'},
 {u'care'}]

In [25]:
review_list[25]

u'This chili powder adds a wonderful spiciness and smokiness to dishes.  Warning, however. Ghost peppers have a spiciness that increases after the first taste, so adjust accordingly.'

In [23]:
map(lambda x:set(removePunctuation(review_list[25]).lower().split(" ")).intersection(set(x.split(" ")).intersection()),TT.topic)

[{u'chili', u'ghost'},
 set(),
 set(),
 set(),
 set(),
 set(),
 {u'wonderful'},
 set(),
 set(),
 set(),
 {u'taste', u'wonderful'},
 {u'adds', u'wonderful'},
 set(),
 set(),
 set(),
 set(),
 {u'chili'},
 set(),
 {u'chili', u'ghost', u'peppers'},
 {u'wonderful'},
 {u'wonderful'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {u'dishes'},
 set()]

In [21]:
TT.topic

[u'hot sauces heat kick hotter pepper drops drop chili wings food burn mustard super habanero dave ghost salsa flavorful insanity',
 u'tea green teas tazo black earl drink bags grey loose cup blend strong orange bag harney iced leaves drinking jasmine',
 u'great price tastes tasting fast snack works makes quick little service day value work job worked way shape goes deal',
 u'coffee cup maker water make filter pot drip drink chicory grounds beans blue mountain morning brew pour ground french cafix',
 u'good pretty quality price value deal tastes bit expected fresh thing little quite looking nice job mix cooking fan easy',
 u'sugar syrup free maple joseph cookies real tastes maltitol low pancakes carb syrups formula corn diabetic mix does tasting diet',
 u'cheese cheeses blue cheddar goat wonderful aged wine crackers creamy igourmet bread milk texture sharp assortment excellent soft mild pound',
 u'product excellent purchase quality used purchased shipping hard exactly products food com

In [27]:
TT.topic_weight[1]

[2.9862853085127044,
 0.5173503443757308,
 0.48502411883062679,
 0.35538087727255796,
 0.34567338922562668,
 0.19113426763161073,
 0.18341024235557535,
 0.18333627630265809,
 0.18213302629732886,
 0.16245170090344285,
 0.14397476395917458,
 0.13285982848969621,
 0.12956828288270855,
 0.124436770887781,
 0.12335532911892066,
 0.12062270619994774,
 0.11494182324577067,
 0.10842787850542597,
 0.10621824238759471,
 0.10377771532244993]